# 📦 Phase 1 — Data Preparation
## Olist Pricing Intelligence

---

**Objectif :** Charger, nettoyer, joindre et enrichir les tables du dataset Olist Brazilian E-commerce pour produire un dataset analytique unique, prêt à l'emploi pour les analyses de pricing (Phase 2) et le dashboard Power BI (Phase 3).

**Dataset source :** [Olist Brazilian E-commerce — Kaggle](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

> ⚠️ **Prérequis :** télécharger les CSV depuis Kaggle et les placer dans `data/raw/` avant d'exécuter ce notebook.

**Tables utilisées :**

| Table | Rôle |
|---|---|
| `olist_orders_dataset` | Commandes, statuts, dates |
| `olist_order_items_dataset` | Prix, fret, produit par ligne |
| `olist_products_dataset` | Caractéristiques produit |
| `olist_product_category_name_translation` | Traduction catégories (PT → EN) |
| `olist_order_reviews_dataset` | Avis et scores clients |
| `olist_sellers_dataset` | Localisation vendeurs |
| `olist_customers_dataset` | État géographique du client |
| `olist_order_payments_dataset` | Mode de paiement, versements |

**Tables exclues volontairement :**

| Table | Raison |
|---|---|
| `olist_geolocation_dataset` | Coordonnées GPS par code postal — hors scope pricing |

**Output :** `data/processed/olist_pricing_dataset.csv` + `olist_pricing_dataset.xlsx`

---

## Table des matières

1. [Imports & Configuration](#1-imports)
2. [Chargement des données](#2-chargement)
3. [Contrôle qualité](#3-qualite)
4. [Nettoyage](#4-nettoyage)
5. [Jointures](#5-jointures)
6. [Feature Engineering](#6-features)
7. [Visualisations de validation](#7-viz)
8. [Export](#8-export)

---
## 1. Imports & Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import warnings
warnings.filterwarnings("ignore")

# ── Style global ──────────────────────────────────────────
plt.rcParams.update({
    "figure.facecolor":  "#F8F9FA",
    "axes.facecolor":    "#FFFFFF",
    "axes.spines.top":   False,
    "axes.spines.right": False,
    "font.family":       "sans-serif",
    "axes.titlesize":    13,
    "axes.titleweight":  "bold",
    "axes.labelsize":    11,
    "xtick.labelsize":   10,
    "ytick.labelsize":   10,
})

# ── Palette Okabe-Ito (daltonisme-friendly, standard scientifique) ──
# Source : Okabe & Ito (2008) — https://jfly.uni-koeln.de/color/
PALETTE = [
    "#E69F00",  # Orange
    "#56B4E9",  # Bleu ciel
    "#009E73",  # Vert
    "#F0E442",  # Jaune
    "#0072B2",  # Bleu foncé
    "#D55E00",  # Vermillon
    "#CC79A7",  # Rose/mauve
    "#000000",  # Noir
]

print("✅ Configuration OK — Palette Okabe-Ito chargée")
print(f"   Couleurs disponibles : {len(PALETTE)}")

# ── Chemins ───────────────────────────────────────────────
DATA_RAW = "../data/raw/"
DATA_OUT = "../data/processed/"
IMG_OUT  = "../outputs/"

os.makedirs(DATA_OUT, exist_ok=True)
os.makedirs(IMG_OUT,  exist_ok=True)

# ── Taux de change ────────────────────────────────────────
# Mettre à jour avec le taux souhaité avant exécution
# Source de référence : https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-brl.en.html
BRL_TO_EUR = 0.16   # 1 R$ = 0.17 € (taux indicatif — à ajuster)

print("✅ Configuration OK")
print(f"   Taux de change appliqué : 1 R$ = {BRL_TO_EUR} €")

✅ Configuration OK
